### Greenscreens.ai homework for python engineer

In [1]:
import numpy as np
import pandas as pd
import random

In this sample script data is taken from the parquet file but in reality the process is the following.
* Customers permanently import their data into our postgres database.
* Everyday we make snapshots in a parquet file for every customer (with all data for every customer). Our goal was to speed up the further reporting and not overload the production database with unnecessary requests. 
    * Special case: we’ve got virtual customers. We call them “jointgs”. They’re not real customers, but just labels assigned to groups of real customers. We need such labels to build aggregated reports and calculate metrics. One real customer could be included in several jointgs. For each such label (jointgs) snapshots are built daily. Jointgs snapshots are created on the fly during select from postgres database based on meta information about jointgs content.
* On our side we have a process of reporting based on customers’ data stored in snapshots. Various reports are launched many times a day based on the data from snapshots.

Problems of current solution:
* Selection of all data for every customer and every jointgs during snapshot preparation is very time consuming
* Data duplication in snapshots because one real customer could be included in several jointgs

You task is to:
* Replace static parquet file with emulated process of importing customer data. We expect you to build a kind of integration test here
* Optimize process of selection using create_date and/or last_updated_date columns
* Get rid of data duplication related to jointgs
* Cover everything with unit tests
* Build sample report (example is in the very bottom)

We provide you with a 100k rows sample. Real database contains about 100 mln rows.


In [2]:
companies_info = pd.read_parquet('meta.prq')
companies_info

,jointgs7,jointgs6,jointgs9,jointgs8,jointgs15,jointgs16,jointgs13,jointgs14,jointgs11,jointgs12,jointgs10,jointgs3,jointgs5,jointgs4,jointGS,jointgs2,companyName
0,false,false,true,false,false,false,true,true,true,true,true,true,false,false,false,false,hafacqafwx
1,false,false,true,false,false,false,true,false,true,true,true,true,false,false,false,false,gkbiszkscy
2,None,false,true,None,false,false,true,true,true,true,true,false,true,false,false,false,hfzzxpajqo
3,None,false,true,None,false,false,true,true,true,true,true,false,false,true,false,None,beyikmvlgc
4,false,false,true,false,false,false,true,true,true,true,true,true,false,false,true,true,cjhysupezw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,false,false,true,false,true,false,true,false,true,true,true,true,false,false,false,false,etkmsiacrc
116,None,false,true,None,false,false,true,true,true,true,true,true,false,false,false,None,nagtfxmbqi
117,false,true,true,false,true,false,true,false,true,true,true,false,false,false,true,false,wftllwxntv
118,false,false,true,false,false,false,true,true,true,true,true,true,false,false,false,false,ubhqnnewtx


In [3]:
database = pd.read_parquet('data.prq')
database.head()

,id,miles,total_linehaul_cost,transport_type,created_date,last_updated_date,currency,exchange_rate,pickup_date,companyName
0,52663811,87.185273,1100.00,N,2023-07-10 11:25:15.288942,2023-11-14 10:09:11.554232,USD,1.0000000,2023-07-08,ktzpaocfcv
1,62665432,178.448448,400.00,N,2024-02-28 15:34:21.045437,2024-02-28 15:34:21.045440,USD,1.0000000,2023-09-27,colejbwjtk
2,51696445,776.475385,1550.00,N,2023-06-13 17:28:11.351722,2023-11-14 17:13:38.321554,USD,1.0000000,2023-06-07,tniauqtfao
3,56809361,570.148827,484.10,D,2023-10-17 12:58:03.011135,2023-11-20 10:59:00.494077,USD,1.0000000,2023-10-13,bzcdosyxok
4,52547945,61.827756,2200.00,R,2023-07-03 18:56:00.543216,2023-11-14 12:36:39.872027,USD,1.0000000,2023-06-20,vfaxzzunlb


In [4]:
# in real life this function selects from postgres database
def get_data(comp):
    if comp[:5] == 'joint':
        company_filter = set(companies_info[companies_info[comp] == 'true'].companyName)
    else:
        company_filter = {comp}

    return database[database.companyName.isin(company_filter)]

In [7]:
# in real life every day creates snapshots for every real company and every jointgs
selected = []
for comp in random.choices([x for x in companies_info.columns.tolist() if x[:5] == 'joint'] + \
                           companies_info.companyName.tolist(), k=10):
    df = get_data(comp)
    df.to_parquet(comp + '.prq', index=False)
    selected.append(comp)

In [16]:
# sample report. please build it for all companies from "selected" list above
# wk - "year-week" based on pickup_date
# volume - number of rows
# median_rate - median of total_linehaul_cost / miles

volume                                          \
wk                         2024-01 2024-02 2024-03 2024-04 2024-05 2024-06   
companyName transport_type                                                   
aauutguybk  D                 12.0     2.0     NaN     6.0     4.0     2.0   
            N                 18.0    12.0    14.0    18.0    18.0    14.0   
            R                 12.0     4.0     2.0     NaN     4.0     2.0   
atugnkiflc  D                  NaN     2.0     1.0     3.0     3.0     3.0   
            N                  7.0    10.0     9.0    10.0     5.0     7.0   
            R                  NaN     2.0     1.0     2.0     NaN     NaN   
bawutedhpu  D                  2.0     5.0     2.0     4.0     2.0     3.0   
            N                  4.0     8.0     6.0    11.0     8.0     7.0   
            R                  3.0     3.0     3.0     1.0     3.0     5.0   
beyikmvlgc  D                 10.0     4.0     4.0     NaN     4.0     2.0   
            N                  2.0    14.0    12.0    14.0    10.0    16.0   
            R                  2.0     6.0     4.0     4.0     NaN     4.0   
blqjyagsse  D                  2.0     2.0     6.0     4.0     2.0     4.0   
            N                  6.0    12.0    10.0    22.0    16.0    22.0   
            R                  4.0     2.0     8.0     6.0     2.0     2.0   
bmgotqfyrg  D                  NaN     2.0     3.0     1.0     1.0     1.0   
            N                  3.0     9.0    10.0     7.0    11.0    11.0   
            R                  2.0     2.0     3.0     3.0     2.0     1.0   
bneonmxxaw  D                  2.0     6.0     4.0     4.0     6.0     2.0   
            N                  8.0    24.0    16.0    18.0     8.0    20.0   
            R                  4.0     2.0     NaN     6.0     2.0     4.0   
bsjhgjllri  D                  2.0     2.0     8.0     2.0     4.0     2.0   
            N                  8.0     8.0    18.0    20.0    12.0    12.0   
            R                 14.0     4.0     4.0     4.0     8.0     2.0   
btnqwknbmr  D                  2.0     6.0     2.0     NaN     6.0     6.0   
            N                 12.0    12.0    12.0    16.0    14.0    12.0   
            R                  NaN     2.0     8.0     NaN     4.0     2.0   
bwiwxhznom  D                  NaN     3.0     1.0     4.0     1.0     2.0   
            N                  3.0     8.0     6.0     6.0     4.0     9.0   
            R                  1.0     1.0     1.0     2.0     2.0     3.0   
byshorhqlg  D                  2.0     4.0     2.0     6.0     NaN     NaN   
            N                  2.0    20.0    20.0    22.0    18.0    18.0   
            R                  4.0     2.0     8.0     2.0     2.0     4.0   
bzcdosyxok  D                  4.0     2.0     4.0     4.0     2.0     NaN   
            N                 20.0    22.0    20.0    20.0    16.0    14.0   
            R                  2.0     4.0     4.0     2.0     4.0     6.0   
cccdemexhc  D                  NaN     2.0     2.0     2.0     8.0     4.0   
            N                 24.0    14.0    20.0    18.0    26.0    16.0   
            R                  4.0     NaN     6.0     2.0     4.0     4.0   
cjhysupezw  D                  4.0     6.0     2.0     8.0     2.0     4.0   
            N                 16.0    22.0     8.0    10.0    14.0    14.0   
            R                  2.0     2.0     2.0     2.0     2.0     2.0   
colejbwjtk  D                  3.0     2.0     4.0     2.0     1.0     2.0   
            N                 10.0     6.0     9.0     7.0     9.0    11.0   
            R                  4.0     2.0     3.0     4.0     4.0     2.0   
csyavfzjwo  D                  4.0     4.0     4.0     NaN     4.0     6.0   
            N                 14.0    26.0    14.0     2.0    18.0    10.0   
            R                  8.0     2.0     6.0     4.0     6.0     2.0   
czuqasseih  D                  NaN    12.0     NaN